In [1]:
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [2]:
# make dictionary from training dataset and keep only most common 3000 words

def make_dict(root_dir):
    all_words = []
    emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
    for mail in emails:
        with open(mail) as m:
            for line in m:
                words = line.split()
                all_words += words;
    dictionary = Counter(all_words)
    list_to_remove = list(dictionary.keys())
    for item in list_to_remove:
        if item.isalpha() == False:   # removed if not alphabet
            del dictionary[item]
        elif len(item) == 1:      # single length character removed
            del dictionary[item]
    dictionary = dictionary.most_common(3000)  # most common 3000 considered
    
    return dictionary

In [3]:
# make use of make_dict and generate a label and word frequency matrix for each example

def extract_features(mail_dir):
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    train_labels = np.zeros(len(files))
    count = 0;
    docID = 0;
    for fil in files:
        with open(fil) as fi:
            for i,line in enumerate(fi):
                if i == 2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i,d in enumerate(dictionary):
                            if d[0] == word :
                                wordD = i
                                features_matrix[docID,wordID] = words .count(word)
            train_labels[docID] = 0;
            filepathTokens = fil.split('/')
            lastToken = filepathTokens[len(filepathTokens)-1]
            if lastToken.startswith("spmsg"):
                train_labels[docID] = 1;
                count += 1;
            docID += 1
    
    return features_matrix, train_labels
        

In [4]:
from sklearn.ensemble import RandomForestClassifier
TRAIN_DIR = "./train-mails"
TEST_DIR = "./test-mails"
dictionary = make_dict(TRAIN_DIR)
features_matrix, labels = extract_features(TRAIN_DIR)
test_feature_matrix, test_labels = extract_features(TEST_DIR)

model = RandomForestClassifier()
#train model
model.fit(features_matrix, labels)

predicted_labels = model.predict(test_feature_matrix)
# accuracy
print (accuracy_score(test_labels, predicted_labels))

1.0


C:\Users\2utka\Anaconda3\envs\DIP\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
